# Conversation Knowledge Graph

This type of memory uses a knowledge graph to recreate memory.


In [4]:
# !/usr/local/bin/python3.9 -m pip install --upgrade pip
# %pip install langchain --quiet
# %pip install openai --quiet
# %pip install python-dotenv --quiet
# %pip install networkx --quiet
# %pip install tiktoken --quiet
# %pip install motor --quiet
# %pip install pymongo --quiet
# %pip install python-dotenv
%pip install -e . 

Obtaining file:///Users/jon/github_repos/jonmatthis/jonbot/scratchpad/langchain_stuff
ERROR: file:///Users/jon/github_repos/jonmatthis/jonbot/scratchpad/langchain_stuff does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Using memory with LLM

In [2]:
from langchain.memory import ConversationKGMemory
from langchain.llms import OpenAI
from pathlib import Path
from dotenv import load_dotenv
import json

from jonbot.backend.data_layer.database.get_or_create_mongo_database_manager import get_mongo_database_manager

load_dotenv()


ModuleNotFoundError: No module named 'jonbot'

In [ ]:
mongo_database_manager = get_mongo_database_manager()

In [ ]:

chat_json_path = Path("/Users/jon/Dropbox/Northeastern/Courses/neural_control_of_real_world_human_movement/2023-09-Fall/classbot_data/backup/2023-10-02_classbot_database.chats.json")

with chat_json_path.open('r') as file:
    chats_og = json.load(file)

chats_= {}

for chat in chats:
    parent_message_id = chat["parent_id"]
    if parent_message_id not in chats_by_student:
        chats_by_student[student_id] = []
    chats_by_student[student_id].append(chat)

print_string = [f"{student_id}: {len(chats)} chats" for student_id, chats in chats_by_student.items()]
print("\n".join(print_string))

Chunk big text into little chunkos

In [ ]:
text_file_name = "sara_shams_and_the_astrolabe_20230917.txt"
big_text = open(text_file_name, 'r').read()
text_split_by_newline = big_text.split("\n")

chunk_lengths_og = [len(chunk) for chunk in text_split_by_newline]

#remove empty chunks
text_split_by_newline = [chunk for chunk in text_split_by_newline if len(chunk) > 1]

chunk_lengths = [len(chunk) for chunk in text_split_by_newline]

print(f"Big text is {len(big_text)} characters long, and split into {len(text_split_by_newline)} lines. The longest line is {max(chunk_lengths)} characters long")

In [ ]:
# combine texts split by newline into chunks of 1000 characters
chunk_size = 2000
text_split_by_newline = [chunk for chunk in text_split_by_newline if len(chunk) > 1]

text_chunks = []
current_chunk = ""
for text in text_split_by_newline:
    if len(current_chunk) + len(text) < chunk_size:
        current_chunk += text + "\n"
    else:
        text_chunks.append(current_chunk)
        current_chunk = text + "\n"

mean_chunk_length = sum([len(chunk) for chunk in text_chunks])/len(text_chunks)
print(f"Combined lines into {len(text_chunks)} chunks of roughly {chunk_size} characters each (mean chunk length is {mean_chunk_length})")

In [ ]:

import tiktoken



def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

tokens_in_string = num_tokens_from_string(big_text, "cl100k_base")

# token prices for GPT-3.5-turbo 4k
# Model	Input	Output
# 4K context	$0.0015 / 1K tokens	$0.002 / 1K tokens

cost_per_token_gpt35  = 0.0015/1000
cost_per_token_gpt4 = 0.03/1000

total_cost_35 = tokens_in_string * cost_per_token_gpt35
total_cost_4 = tokens_in_string * cost_per_token_gpt4

print(f"Text is {tokens_in_string} tokens long, and costs ${total_cost_35} to input into GPT-3.5-turbo, and ${total_cost_4} to input into GPT-4")


#  Stuff the story into the memory, one chunk at a time

In [ ]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.memory import ConversationEntityMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationChain

llm = ChatOpenAI(temperature=0,
                 model_name="gpt-3.5-turbo",
                 callbacks=[StdOutCallbackHandler()],
                 streaming = True)

entity_memory = ConversationEntityMemory(llm=llm)
conversation_chain = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=entity_memory
)

human_input = "I am going to tell you the story of Sara Shams and the Astrolabe written by Paul Matthis. It is a long story, so I will tell it to you in chunks. When I tell you each chunk, you will ask me what happens next. I will then tell you what happens next. We will continue this until I tell you that the story is over. Then you will tell me what happened in the story."
print(conversation_chain.run(input=human_input ))

print("KG memory is set up!")

In [ ]:
# conversation_with_kg.predict(input="Who is Paul Matthis?")

In [ ]:
# conversation_with_kg.predict(input="What are we doing here? What's the plan?")

In [ ]:

use_chunks = -1

for chunk_number, chunk in enumerate(text_chunks[:use_chunks]):
    print(f"Inputting chunk {chunk_number} of {len(text_chunks[:use_chunks])}")
    print(conversation_chain.run(input=chunk))
    print("----------------\n"
          "----------------")

human_input = "That's all for now, I'll tell you more later."
conversation_chain.run(input=human_input)



In [ ]:
print(conversation_chain.run(input ="what is this story about?"))

In [ ]:
entity_memory.load_memory_variables({"input": "who is Sara?"})


In [ ]:
conversation_chain.predict(input="Who are the characters in this story?")

In [ ]:
from pprint import pprint

pprint( entity_memory.entity_store.store, indent=4)

In [ ]:
entity_memory.load_memory_variables({"input": "who is the author of this book?"})